# Tariff rates
Williams, H. Paul. Model building in mathematical programming. John Wiley & Sons, 2013.

In [1]:
from typing import Dict

import numpy as np
import pandas as pd
from pulp import LpProblem, LpMaximize, LpMinimize, LpVariable, lpSum

## 集合

In [2]:
periods = np.arange(5)
types = np.arange(3)

In [19]:
demands = [15000, 30000, 25000, 40000, 27000]  # for each period
n_machines = [12, 10, 5] # for each type
n_hours = np.array([6, 3, 6, 3, 6])

In [4]:
prod_lb = np.array([850, 1250, 1500])
prod_ub = np.array([2000, 1750, 4000])
cost_min = np.array([1000, 2600, 3000])
cost_unit = np.array([2, 1.3, 3])
cost_start = np.array([2000, 1000, 500])

In [26]:
n_worked = pd.Series(LpVariable.dicts("n_worked", types, cat='Integer'))

In [27]:
n_working = pd.DataFrame({t: {p: LpVariable(f"n_working_{t}_{p}", lowBound=0, upBound=n_machines[t], cat="Integer")
                              for p in periods} for t in types})

In [28]:
prod_additional = pd.DataFrame({t: {p: LpVariable(f"prod_additional_{t}_{p}", lowBound=prod_lb[t], upBound=prod_ub[t])
                                    for p in periods} for t in types})

In [29]:
capacity_rest = n_working * (prod_ub - prod_lb) - prod_additional

In [30]:
cost = (n_working * cost_min * n_hours[:, None] + prod_additional * cost_unit + n_worked * cost_start).values.sum()

In [31]:
model = LpProblem(name="TariffRates", sense=LpMinimize)
model += cost

In [32]:
for p in periods:
    for t in types:
        model += n_worked[t] >= n_working.loc[p, t]
    model += capacity_rest.sum(axis=1)[p] >= 0
    model += (n_working * prod_lb + prod_additional).sum(axis=1)[p] >= demands[p]
    model += (n_working * prod_ub).sum(axis=1)[p] >= 1.15 * demands[p]

In [33]:
model.solve()

-1

In [34]:
cost.value()

952709.25